In [1]:
import pandas as pd

In [2]:
sales_data = {
    'customer_id':('A','A','A','A','A','A','B','B','B','B','B','B','C','C','C'),
    'order_date':('2021-01-01','2021-01-01','2021-01-07','2021-01-10','2021-01-11','2021-01-11',
                 '2021-01-01','2021-01-02','2021-01-04','2021-01-11','2021-01-16','2021-02-01',
                 '2021-01-01','2021-01-01','2021-01-07'),
    'product_id':(1,2,2,3,3,3,2,2,1,1,3,3,3,3,3)
}

menu_data = {
    'product_id':(1,2,3),
    'product_name':('sushi','curry','ramen'),
    'price': (10,15,12)
}

member_data = {
    'customer_id':('A','B'),
    'join_date':('2021-01-07', '2021-01-09')
}

In [3]:
sales_df = pd.DataFrame(sales_data)
menu_df = pd.DataFrame(menu_data)
member_df = pd.DataFrame(member_data)

In [4]:
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
member_df['join_date'] = pd.to_datetime(member_df['join_date'])

In [5]:
menu_df

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [6]:
sales_df

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [7]:
sales_df.merge(menu_df).drop('product_id',axis=1).groupby(['customer_id']).sum().rename(columns={'price':'total_spent'})

,total_spent
customer_id,
A,76
B,74
C,36
